# Hands-on Lab: Interactive Visual Analytics with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using Folium.

# Objectives:

This lab contains the following tasks:

- TASK 1: Mark all launch sites on a map
- TASK 2: Mark the success/failed launches for each site on the map
- TASK 3: Calculate the distances between a launch site to its proximities After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
%pip install -q folium pandas



Note: you may need to restart the kernel to use updated packages.


In [2]:
import io
import math
import pandas as pd
import folium

from folium.plugins import MarkerCluster
from folium.features import DivIcon
from folium import LatLngPopup



## Task 1: Mark all launch sites on a map

In [3]:
# Load .csv data set
geo_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

spacex_df = pd.read_csv(geo_url)

# Confirm data set loaded

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


First, let's try to add each site's location on a map using site's latitude and longitude coordinates

In [4]:
# From the data set, look at the co-ordinates for each site.
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [6]:
# NASA JSC marker
circle = folium.Circle(
    nasa_coordinate,
    radius=10000,
    color="#d35400",
    fill=True
).add_child(folium.Popup("NASA Johnson Space Center"))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size:12px;color:#d35400;"><b>NASA JSC</b></div>',
    )
)

site_map.add_child(circle)
site_map.add_child(marker)

site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

In [7]:
for _, row in launch_sites_df.iterrows():
    coord = [row["Lat"], row["Long"]]
    name = row["Launch Site"]

    folium.Circle(
        coord,
        radius=1000,
        color="#d35400",
        fill=True,
        fill_opacity=0.5,
        popup=name
    ).add_to(site_map)

    folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(200, 18),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px;color:#d35400;"><b>{name}</b></div>'
        )
    ).add_to(site_map)

# Optional: ensure all launch sites are in view
site_map.fit_bounds(launch_sites_df[["Lat", "Long"]].values.tolist())

site_map


# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Let's first create a MarkerCluster object

In [9]:
marker_cluster = MarkerCluster()

TODO: Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

In [10]:
# Ensure class is numeric 0/1
spacex_df["class"] = pd.to_numeric(spacex_df["class"], errors="coerce").fillna(0).astype(int)

# Create marker_color
spacex_df["marker_color"] = spacex_df["class"].map({1: "green", 0: "red"}).fillna("red")

spacex_df["class"].value_counts(), spacex_df["marker_color"].value_counts()


(class
 0    32
 1    24
 Name: count, dtype: int64,
 marker_color
 red      32
 green    24
 Name: count, dtype: int64)

TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

In [11]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=folium.Icon(color=row["marker_color"], icon="info-sign"),
        popup=f'Launch Site: {row["Launch Site"]}<br>Class: {row["class"]}'
    ).add_to(marker_cluster)

site_map


# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [15]:
from folium.plugins import MousePosition

formatter = "function(num) {return L.Util.formatNum(num, 5);};"

MousePosition(
    position="topright",
    separator=" | ",
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(site_map)

site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Great-circle distance between two points (lat/lon) in kilometers.
    """
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = (math.sin(dlat / 2) ** 2) + math.cos(lat1) * math.cos(lat2) * (math.sin(dlon / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


TODO: Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [18]:
launch_site_name = launch_sites_df.loc[0, "Launch Site"]  # or set explicitly
launch_row = launch_sites_df[launch_sites_df["Launch Site"] == launch_site_name].iloc[0]

launch_lat = float(launch_row["Lat"])
launch_lon = float(launch_row["Long"])

launch_site_name, launch_lat, launch_lon


('CCAFS LC-40', 28.56230197, -80.57735648)

In [21]:
target_name = "Coastline point"
target_lat = 28.56265
target_lon = -80.5679

distance_km = calculate_distance(launch_lat, launch_lon, target_lat, target_lon)
distance_km


0.924350067980974

In [28]:
# Recreate map fresh for distance visualization
site_map = folium.Map(
    location=[launch_lat, launch_lon],
    zoom_start=14,
    tiles="OpenStreetMap"
)

# Label marker at target
folium.Marker(
    location=[target_lat, target_lon],
    icon=DivIcon(
        icon_size=(260, 36),
        icon_anchor=(0, 0),
        html=f"<div style='font-size:14px;color:#d35400;'><b>{target_name}: {distance_km:.2f} KM</b></div>",
    ),
).add_to(site_map)

# Draw line from launch site to target
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [target_lat, target_lon]],
    weight=3
).add_to(site_map)

site_map


TODO: Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use MousePosition to find the their coordinates on the map first

In [32]:
# Launch site coordinate 28.56230197, -80.57735648
launch_lat = 28.56230197
launch_lon = -80.57735648

# Fresh map
site_map = folium.Map(
    location=[launch_lat, launch_lon],
    zoom_start=13,
    tiles="OpenStreetMap"
)

# Mark the launch site
folium.Marker(
    [launch_lat, launch_lon],
    popup="CCAFS LC-40",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

site_map


In [33]:
from folium.plugins import MousePosition

formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position="topright",
    separator=" | ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


In [34]:
# Replace with coordinates you found using MousePosition
prox_lat = 28.53439
prox_lon = -81.3834

# Distance (km)
distance_km = calculate_distance(launch_lat, launch_lon, prox_lat, prox_lon)

# Marker showing distance text at the proximity point
folium.Marker(
    [prox_lat, prox_lon],
    icon=DivIcon(
        icon_size=(260, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:14px;color:#d35400;"><b>Proximity: {distance_km:.2f} KM</b></div>'
    )
).add_to(site_map)

# Line from launch site to proximity point
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [prox_lat, prox_lon]],
    color="blue",
    weight=3
).add_to(site_map)

site_map
